In [1]:
from vkapi.methods import *
from vkapi import AccessToken

import itertools
import time

Lets collect all the members of some group. [**VK Music**](https://vk.com/vkmusic) seems good.

In [2]:
def get_users(gid):
    offset = 0
    total_uids = []
    while True:
        uids = groups.getMembers(group_id=gid, offset=offset)['users']
        if not len(uids):
            break
        total_uids.extend(uids)
        offset += 1000
        time.sleep(0.2)
    return total_uids

In [3]:
%%time
music_users = get_users(147845620)

CPU times: user 1.55 s, sys: 122 ms, total: 1.67 s
Wall time: 21.7 s


In [4]:
print("Total: {} users".format(len(music_users)))

Total: 49844 users


### `execute`

Algorithm code in **VKScript** — a format similar to **JavaSсript** or **ActionScript** and assuming compatibility with **ECMAScript**. 

Separate operators by semicolons. Terminate the algorithm with the command return %expression%. 

**Code** may contain up to 25 API requests.

https://oauth.vk.com/authorize?client_id=YOUCLIENTID&display=page&redirect_uri=oauth.vk.com/blank&scope=&response_type=token

In [5]:
AccessToken.token = 'HERE IS THE TOKEN'

In [6]:
code_pattern = '''
var nReq = 0; var nReqMax = 25;
var gid = %s; var offset = %s;
var users = [];
while (nReq < nReqMax) {
    var new = API.groups.getMembers({"offset":offset, "group_id":gid})["users"];
    if (new.length == 0) return {"offset": offset, "users":users};
    users.push(new);
    offset = offset + 1000; nReq = nReq + 1;
}
return {"offset":offset, "users":users};'''

In [7]:
def get_users_execute(gid):
    offset = 0
    total_users = []
    while True:
        res = execute(code_pattern % (gid, offset))
        new_users = [a for a in itertools.chain(*res['users'])]
        if not len(new_users): 
            break
        total_users.extend(new_users)
        offset = res['offset']
        time.sleep(0.2)
    return total_users

In [8]:
%%time
music_users_execute = get_users_execute(147845620)

CPU times: user 132 ms, sys: 14 ms, total: 146 ms
Wall time: 3.69 s


In [9]:
print("Total: {} users".format(len(music_users_execute)))

Total: 49844 users


|Method      |time, s     | requires token |
|------------|------------|----------------| 
|usual API   |21.7        |NO              |
|`execute`   |3.7         |YES             |